In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime
from sklearn.model_selection import train_test_split

# Phase 2: Data preprocessing

## First we redo the data changes from the 1st phase

In [2]:
connections, devices, processes, profiles = pd.read_csv('data/connections.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/devices.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/processes.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/profiles.csv', sep='\t', keep_default_na=False, na_values='')

Iterative way to redo the changes:

In [5]:
def get_outliers(column: pd.Series):
    lower_quartile = column.quantile(0.25)
    upper_quartile = column.quantile(0.75)
    iqr = upper_quartile - lower_quartile
    return column[(column < lower_quartile - 1.5*iqr) | (column > upper_quartile + 1.5*iqr)]


In [ ]:
def iterative_reformat(processes_ptr: pd.DataFrame, connections_ptr: pd.DataFrame) -> pd.DataFrame:
    connections_ptr['ts'] = pd.to_datetime(connections_ptr['ts'])
    processes_ptr['ts'] = pd.to_datetime(processes_ptr['ts'])
    connections_ptr['mwra'] = connections_ptr['mwra'].astype(bool)
    processes_ptr['mwra'] = processes_ptr['mwra'].astype(bool)
    merged = processes_ptr.merge(connections_ptr, on=['ts', 'imei', 'mwra'], how='inner')
    # handle null values and outliers
    for column in merged.columns:
        # if more than 5% are NaN values or more than 5% are outliers, we don't use that column
        column_outliers = get_outliers(merged[column])
        if ((merged[column].dropna().shape[0]/merged.shape[0] > 0.05) or 
            (column_outliers.shape[0] / merged.shape[0] > 0.05)):
            merged.drop(columns=column)
            continue
        # if there are some null values, we replace the data that's neutral in respect to mwra
        if merged[column].isnull().any():
            # we get means of the distributions for rows with present and non-present malware related activity
            means_per_mwra = merged.groupby('mwra')[column].mean()
            # we average those means, meaning the manufactured value won't be likely to affect predicted mwra 
            imputed_value = means_per_mwra.mean()
            merged[column].fillna(imputed_value, inplace=True)
        #  if there are any outliers, we replace them with the edge values
        if column_outliers.shape[0]:
            iqr = stats.iqr(merged[column])
            lower_limit = merged[column].quantile(0.25)  - 1.5 * iqr
            upper_limit = merged[column].quantile(0.75)  - 1.5 * iqr
            merged[column] = merged[column].apply(
                lambda x: lower_limit if x < lower_limit else upper_limit if x > upper_limit else x
            )
    return merged

# Phase 2-1: Realizing data preprocessing

## 2-1a & 2-1b
Splitting the data into training and testing sets + transforming data for ML

First we create a combined table for data to work with. As we learnt in the previous phase, we will use only connections and processes tables. Devices and profiles couldn't be connected logically with the other two tables. That's because there were multiple profiles/devices per imei. And it wasn't a fixed amount of profiles/devices per imei either, so we can't just make a column for all locations/usernames/etc. Even if we did that, there wasn't a correlation found between any of the columns in these tables and mwra.

In [7]:
combined_table = iterative_reformat(processes, connections)

,ts,imei,mwra,p.android.chrome,p.android.documentsui,p.android.gm,p.system,p.android.packageinstaller,p.android.settings,p.android.externalstorage,...,c.android.youtube,c.dogalize,c.android.gm,c.katana,c.android.chrome,c.raider,c.android.vending,c.UCMobile.intl,c.UCMobile.x86,c.updateassist
0,2018-05-05 10:00:00,3590433799317661107,False,7.41473,10.17656,14.80917,12.14702,11.55620,8.33912,14.02450,...,11.65403,10.65335,10.48791,16.91620,5.04564,36.21508,14.24956,4.09319,15.20934,5.11764
1,2018-05-05 10:01:00,3590433799317662063,True,9.40603,6.57378,6.06519,10.56643,16.74062,13.78434,7.57297,...,10.29551,10.46363,10.98438,15.65637,14.82931,24.83765,57.49911,23.94919,8.82448,44.13570
2,2018-05-05 10:02:00,863033069630348313,False,13.61225,11.73312,8.99679,10.79425,12.60312,10.78121,11.90788,...,12.12831,9.53752,6.78080,6.98948,10.92433,3.29441,20.37891,52.55353,49.51037,99.75215
3,2018-05-05 10:03:00,359043379931766114,True,8.14970,9.53996,8.28249,10.80629,13.97670,12.73047,11.64714,...,11.06419,11.55759,8.87744,11.84499,15.45902,24.78878,36.40299,25.77644,21.84167,65.17774
4,2018-05-05 10:04:00,3590433799317661842,True,8.20358,8.94156,8.62248,8.33003,13.44049,13.31239,11.40689,...,12.08457,10.42860,9.82241,12.48869,12.85363,40.98237,74.13316,96.77450,18.48116,90.77304


now onto splitting the data into testing and training

In [8]:
# we separate the features and the target
X = combined_table.drop(columns=['mwra'])
y = combined_table['mwra']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

As we didn't use any non-numerical data, we don't need to do any conversions to numerical. Most of the data had huge cardinality either way which would increase likelihood of overfitting and difficulty of encoding. 

Example for what we would do if we were to use the categorical data from profiles and devices table

In [23]:
# if the cardinality was too high to use one hot encoding, we can hash the values and now they are numbers
mail_encoded = profiles['mail'].apply(lambda x: hash(x))
profiles['mail'].nunique(), mail_encoded.nunique()
# if one hot encoding was feasible, it could be doable like this
continents = devices["location"].apply(lambda x: x.split('/')[0])
continents.head()

0      America
1    Australia
2       Europe
3       Europe
4      America
Name: location, dtype: object